In [100]:
# # reading harvardx data from the csv
# # still a work in progress since their dataset is malformatted
# import csv

# # load data
# csvfile_path = '../datasets/corpus_HarvardX_LatestCourses_based_on_2016-10-18.csv'

# with open(csvfile_path, 'r') as csvfile:
#     reader = csv.DictReader(csvfile)
#     print reader.next()

In [37]:
import search
reload(search)

# reopen index, but reload it instead of creating it afresh
udacity_se = search.UdacitySearchEngine(create=False)
    
results = udacity_se.search("neural networks")
print results
# BUG: can't do anything with this results object because the reader is closed!!

<Top 2 Results for And([Or([Term('title', u'neural'), Term('subtitle', u'neural'), Term('expected_learning', u'neural'), Term('syllabus', u'neural'), Term('summary', u'neural'), Term('short_summary', u'neural')]), Or([Term('title', u'networks'), Term('subtitle', u'networks'), Term('expected_learning', u'networks'), Term('syllabus', u'networks'), Term('summary', u'networks'), Term('short_summary', u'networks')])]) runtime=0.0095751285553>


In [20]:
results

ReaderClosed: 